In [1]:
from gbmi.exp_modular_fine_tuning.train import MODULAR_ADDITION_113_CLOCK_CONFIG
from gbmi.exp_modular_fine_tuning.train import MODULAR_ADDITION_113_PIZZA_CONFIG
from gbmi.exp_modular_fine_tuning.train import ModularFineTuningTrainingWrapper
from gbmi.model import train_or_load_model
import torch
import einops
from torch import tensor
from math import *
device = 'cuda'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
config_clock = MODULAR_ADDITION_113_CLOCK_CONFIG
config_pizza = MODULAR_ADDITION_113_PIZZA_CONFIG
runtime_clock, model_clock = train_or_load_model(config_clock)
runtime_pizza, model_pizza = train_or_load_model(config_pizza)
model_clock.to(device)
model_pizza.to(device)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Could not load model gbmi/ModularFineTuning-3-25000-epochs-attention-rate-1/ModularFineTuning-3-25000-epochs-attention-rate-1-0042bf0daaddb6a35d03ad309fcb48bf:latest from wandb:
 'NoneType' object has no attribute 'get'


wandb: Currently logged in as: jasongross (gbmi). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data_train.shape: torch.Size([3830, 3]), data_test.shape: torch.Size([8939, 3])


...

Saving to disk...
Saving to WandB...


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_loss,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
acc,1.0
epoch,25000
loss,1e-05
test_acc,0.29623
test_loss,29.31886


Moving model to device:  cuda
Moving model to device:  cuda


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (ln2): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)

In [6]:
def loss_fn(logits,labels):
    print(logits.shape)
    if len(logits.shape)==3:
        logits = logits[:,:, -1]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()

In [7]:
a_vector = einops.repeat(torch.arange(113), "i -> (i j)", j=113)
b_vector = einops.repeat(torch.arange(113), "j -> (i j)", i=113)
equals_vector = einops.repeat(torch.tensor(113), " -> (i j)", i=113, j=113)
dataset = torch.stack([a_vector, b_vector, equals_vector], dim=1).to(device)
print(dataset.shape)

NameError: name 'p' is not defined

In [ ]:
optim_pizza = torch.optim.AdamW(model_pizza).parameters())
optim_clock = torch.optim.AdamW(model_clock).parameters())


In [ ]:
def run_batch(x):
        labels = (x[:, 0] - x[:, 1]) % p

        y_preds_pizza = model_pizza.run_with_hooks(x)
        y_preds_clock = model_clock(x)

        loss_pizza = loss_fn(y_preds_pizza, labels)
        loss_clock = loss_fn(y_preds_clock, labels)
        loss_clock.backward()
        loss_pizza.backward()
        optim_pizza.step()
        optim_pizza.zero_grad()
        optim_clock.step()
        optim_clock.zero_grad()
        return loss
run_batch(dataset)